<a href="https://colab.research.google.com/github/JoaquinGonzalezSimon/Data_science_and_ML_from_Medium/blob/main/220930_Predict_Car_Prices_with_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project was taken from the following link and I use it to practice my skills

https://thecleverprogrammer.com/2020/09/29/fake-currency-detection-with-machine-learning/

https://medium.com/coders-camp/180-data-science-and-machine-learning-projects-with-python-6191bc7b9db9

In [2]:
pip install jovian

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.0 MB/s 
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6503 sha256=a4e8712d7debf9eb0d930acce7e2d335282352ced6a2677596df3d39f4ea6259
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import torch
import jovian
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [32]:
data_raw = pd.read_csv('https://raw.githubusercontent.com/amankharwal/Website-data/master/car%20data.csv')

In [33]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [34]:
data_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [35]:
len(data_raw)

301

In [36]:
int(ord('Alejandro'[2]))

101

In [37]:
your_name = 'Aman Kharwal'
def customize_dataset(dataframe_raw, rand_str):
    data = dataframe_raw.copy()
    data = data.sample(int(0.95*len(data)), random_state = int(ord(rand_str[0])))
    data.Year = data.Year * ord(rand_str[1])/100
    data.Selling_Price = data.Selling_Price * ord(rand_str[2])/100
    data = data.drop(['Car_Name'], axis=1)
    return data

In [38]:
data = customize_dataset(data_raw, your_name)

In [39]:
data.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
2,2198.53,7.0325,9.85,6900,Petrol,Dealer,Manual,0
128,2198.53,0.7760,0.87,3000,Petrol,Individual,Manual,0
286,2197.44,5.4805,7.90,28569,Petrol,Dealer,Manual,0
272,2196.35,7.2750,10.00,27600,Petrol,Dealer,Manual,0
205,2197.44,5.0925,5.70,3493,Petrol,Dealer,Manual,1


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 2 to 147
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           285 non-null    float64
 1   Selling_Price  285 non-null    float64
 2   Present_Price  285 non-null    float64
 3   Kms_Driven     285 non-null    int64  
 4   Fuel_Type      285 non-null    object 
 5   Seller_Type    285 non-null    object 
 6   Transmission   285 non-null    object 
 7   Owner          285 non-null    int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 20.0+ KB


In [61]:
data.Selling_Price

2       7.0325
128     0.7760
286     5.4805
272     7.2750
205     5.0925
        ...   
15      7.5175
98      6.8385
64     32.0100
196     0.1649
147     0.5335
Name: Selling_Price, Length: 285, dtype: float64

In [62]:
input_cols = data._get_numeric_data().drop('Selling_Price', axis=1).columns.to_list()
cat_cols = data.select_dtypes('object').columns.to_list()
output_col = ['Selling_Price']

In [63]:
def dataframe_to_arrays(dataframe):
    dataframe1 = dataframe.copy(deep=True)
    for col in cat_cols:
      dataframe1[col] = dataframe1[col].astype('category').cat.codes
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_col].to_numpy()
    return inputs_array, targets_array

In [64]:
inputs_array, targets_array = dataframe_to_arrays(data)

In [65]:
inputs_array

array([[2.19853e+03, 9.85000e+00, 6.90000e+03, 0.00000e+00],
       [2.19853e+03, 8.70000e-01, 3.00000e+03, 0.00000e+00],
       [2.19744e+03, 7.90000e+00, 2.85690e+04, 0.00000e+00],
       ...,
       [2.19853e+03, 3.62300e+01, 6.00000e+03, 0.00000e+00],
       [2.18872e+03, 5.20000e-01, 5.00000e+05, 0.00000e+00],
       [2.19635e+03, 8.40000e-01, 5.80000e+04, 0.00000e+00]])

In [66]:
targets_array

array([[ 7.0325],
       [ 0.776 ],
       [ 5.4805],
       [ 7.275 ],
       [ 5.0925],
       [ 0.4656],
       [ 2.7645],
       [ 0.3395],
       [ 1.6975],
       [ 4.8015],
       [ 8.9725],
       [ 5.141 ],
       [ 3.783 ],
       [ 8.827 ],
       [ 4.268 ],
       [ 3.3853],
       [19.1575],
       [ 7.0325],
       [10.9125],
       [12.513 ],
       [14.2881],
       [ 4.753 ],
       [ 5.0925],
       [ 3.1525],
       [ 7.2265],
       [ 0.5335],
       [ 2.813 ],
       [ 0.3007],
       [ 0.2619],
       [ 7.178 ],
       [ 0.582 ],
       [ 5.9655],
       [ 2.7645],
       [ 8.148 ],
       [ 0.485 ],
       [ 7.275 ],
       [ 0.9215],
       [ 5.7715],
       [ 4.365 ],
       [ 0.485 ],
       [ 4.365 ],
       [ 2.91  ],
       [ 0.4365],
       [ 0.291 ],
       [ 4.6075],
       [ 0.2425],
       [ 0.4074],
       [ 3.977 ],
       [ 4.365 ],
       [ 0.4365],
       [ 0.4656],
       [ 1.0185],
       [ 2.813 ],
       [19.3903],
       [ 0.4365],
       [ 0

In [67]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

In [73]:
inputs.shape, targets.shape

(torch.Size([285, 4]), torch.Size([285, 1]))

In [69]:
dataset = TensorDataset(inputs, targets)

In [70]:
dataset

In [75]:
train_ds, vals_ds = random_split(dataset, [228,57])
batch_size = 128

In [76]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(vals_ds, batch_size)

In [94]:
input_size = len(input_cols)
output_size = len(output_col)

class CarsModel(nn.Module):
    def __init__(self):
      super().__init__()
      self.linear = nn.Linear(input_size, output_size)

    def forward(self, xb):
      out = self.linear(xb)
      return out

    def training_step(self, batch):
      inputs, targets = batch
      out = self(inputs)
      loss = F.l1_loss(out, targets)
      return loss

    def validation_step(self, batch):
      inputs, targets = batch
      out = self(inputs)
      loss = F.l1_loss(out,targets)
      return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
      batch_losses = [x['val_loss'] for x in outputs]
      epoch_loss = torch.stack(batch_losses).mean()
      return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
      if (epoch+1) % 20 == 0 or epoch == num_epochs -1:
        print('Epoch [{}], val_loss: {:.4f}'.format(epoch+1, result['val_loss']))

In [95]:
model = CarsModel()
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0125, -0.2317,  0.1048, -0.0325]], requires_grad=True),
 Parameter containing:
 tensor([-0.4559], requires_grad=True)]

In [96]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [97]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [98]:
result = evaluate(model, val_loader)
result

{'val_loss': 3955.63037109375}

In [99]:
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 3389.7048
Epoch [40], val_loss: 2821.3862
Epoch [60], val_loss: 2256.1663
Epoch [80], val_loss: 1689.3011
Epoch [90], val_loss: 1406.6409


In [101]:
epochs = 50
lr = 1e-7
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 99.7965
Epoch [40], val_loss: 82.9036
Epoch [50], val_loss: 47.8342


In [102]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print('Input: ', input)
    print('Target: ', target)
    print('Prediction: ', prediction)

In [103]:
input, target = vals_ds[0]

In [104]:
predict_single(input, target, model)

Input:  tensor([2.1964e+03, 8.4000e-01, 2.3000e+04, 0.0000e+00])
Target:  tensor([0.4656])
Prediction:  tensor([-19.7858])


In [106]:
input, target = vals_ds[10]
predict_single(input, target, model)

Input:  tensor([2198.5300,    3.6000, 2135.0000,    0.0000])
Target:  tensor([2.7645])
Prediction:  tensor([6.8881])
